In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from scipy.linalg import eigh, qr

from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton

from susy_qm import calculate_Hamiltonian2 

In [78]:
potential = "AHO"
shots = 1024
cutoff = 4

    
H = calculate_Hamiltonian2(cutoff, potential)
eigvals, eigvecs = eigh(H)
num_qubits = int(1 + np.log2(cutoff))
print(num_qubits)

3


In [58]:
# Diagonalize H
eigvals, eigvecs = eigh(H)

# Setup
shots = 1024
sim = AerSimulator(method="automatic", seed_simulator=42)

def prepare_state(params):
    qc = QuantumCircuit(num_qubits)
    qc.x(0)
    qc.ry(params[0], 0)
    qc.ry(params[1], 1)
    qc.ry(params[2], 2)
    qc.ry(params[3], 3)
    
    return qc

def projector_unitary(eigvec):
    """Return U† that maps |v⟩ → |0⟩"""
    # Use QR to generate full unitary with eigvec as first column
    basis = [eigvec] + [np.eye(len(eigvec))[:, i] for i in range(1, 2**num_qubits)]
    U = np.linalg.qr(np.column_stack(basis))[0]
    return U.conj().T  # U†

def compute_expval_shots(params):
    total = 0.0
    for j in range(len(eigvals)):
        λ = eigvals[j]
        v = eigvecs[:, j]

        # Create circuit ψ(θ)
        qc = prepare_state(params)

        # Apply U† to rotate to computational basis
        U_dag = projector_unitary(v)
        qc.unitary(U_dag, list(range(num_qubits)))

        # Measure
        qc.measure_all()

        # Transpile + run
        tqc = transpile(qc, sim)
        result = sim.run(tqc, shots=shots).result()
        counts = result.get_counts()

        # Probability of measuring |000>
        p_zero = counts.get("0" * num_qubits, 0) / shots
        total += λ * p_zero

    return total


In [59]:
seed=42

num_params = 4
bounds = [(0, 2 * np.pi) for _ in range(num_params)]

max_iter = 1024
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

halton_sampler = Halton(d=num_params, seed=seed)
scaled_samples = 2 * np.pi * halton_sampler.random(n=popsize)


result = differential_evolution(
    compute_expval_shots,
    bounds=bounds,
    maxiter=max_iter,
    tol=tol,
    atol=abs_tol,
    strategy=strategy,
    popsize=popsize,
    init=scaled_samples,
    seed=seed
)

In [60]:
result.success

True

In [61]:
result.fun

0.03197885013614109

In [62]:
eigvals[0]

0.03201011000919694

In [72]:
eigvals, eigvecs = eigh(H)
n = int(np.log2(H.shape[0]))
shots = 1024
sim = AerSimulator(method='automatic', seed_simulator=42)

# Generate eigenprojector basis unitaries
projector_data = []
for i in range(len(eigvals)):
    eigval = eigvals[i]
    eigvec = eigvecs[:, i]
    cols = [eigvec] + [np.eye(2**n)[:, j] for j in range(1, 2**n)]
    U = qr(np.column_stack(cols))[0]
    projector_data.append((eigval, U.conj().T))

# Create parameterized ansatz once
params = [Parameter(f'theta_{i}') for i in range(n)]

def base_ansatz():
    qc = QuantumCircuit(n)
    for i in range(n):
        qc.ry(params[i], i)
    return qc

ansatz_template = base_ansatz()

# -----------------------------
# Cost Function
# -----------------------------

def compute_expval_shots(param_values):
    total = 0.0
    bindings = dict(zip(params, param_values))

    for eigval, Udag in projector_data:
        qc = ansatz_template.assign_parameters(bindings, inplace=False)
        qc = qc.copy()
        qc.unitary(Udag, range(n))
        qc.measure_all()

        tqc = transpile(qc, sim)
        result = sim.run(tqc, shots=shots).result()
        counts = result.get_counts()
        prob_0 = counts.get('0' * n, 0) / shots
        total += eigval * prob_0

    return total


In [2]:
potential = "AHO"
shots = 1024
cutoff = 4 
H = calculate_Hamiltonian2(cutoff, potential)

In [9]:
eigvals, eigvecs = eigh(H)
n = int(np.log2(H.shape[0]))
shots = 1024
sim = AerSimulator(method="automatic", seed_simulator=42)

# Parameters for ansatz
params = [Parameter(f'theta_{i}') for i in range(n)]

# Build parameterized ansatz once
def build_ansatz():
    qc = QuantumCircuit(n)
    for i in range(n):
        qc.ry(params[i], i)
    return qc

ansatz_template = build_ansatz()

# Precompute projectors (U† circuits) once
projector_circuits = []
for i in range(len(eigvals)):
    eigval = eigvals[i]
    eigvec = eigvecs[:, i]
    cols = [eigvec] + [np.eye(2**n)[:, j] for j in range(1, 2**n)]
    U = qr(np.column_stack(cols))[0]
    U_dag = U.conj().T

    qc = ansatz_template.copy()
    qc.unitary(U_dag, range(n))
    qc.measure_all()
    projector_circuits.append((eigval, qc))

# Expectation value function using finite shots
def compute_expval(params_val):
    bindings = dict(zip(params, params_val))
    bound_circuits = [qc.assign_parameters(bindings, inplace=False) for _, qc in projector_circuits]
    transpiled = transpile(bound_circuits, backend=sim)
    result = sim.run(transpiled, shots=shots).result()

    energy = 0.0
    for i, (eigval, _) in enumerate(projector_circuits):
        counts = result.results[i].data.counts
        counts = {k[::-1]: v for k, v in counts.items()}  # bitstring reversal for Qiskit's endian convention
        #prob_0 = counts.get("0" * n, 0) / shots
        prob_0 = counts.get('0x0', 0) / shots
        energy += eigval * prob_0

    return energy

In [10]:
seed=42

num_params = n
bounds = [(0, 2 * np.pi) for _ in range(num_params)]

max_iter = 1000
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

halton_sampler = Halton(d=num_params, seed=seed)
scaled_samples = 2 * np.pi * halton_sampler.random(n=popsize)


result = differential_evolution(
    compute_expval,
    bounds=bounds,
    maxiter=max_iter,
    tol=tol,
    atol=abs_tol,
    strategy=strategy,
    popsize=popsize,
    init=scaled_samples,
    seed=seed
)

In [11]:
result.fun

-0.16478526068502206

In [12]:
eigvals

array([-0.16478526,  0.6733101 ,  1.66794264,  2.49907548, 14.4516899 ,
       15.28978526, 23.62592452, 24.45705736])